In [58]:
import torch
import numpy as np

def _2div(arr):
    arr_res = arr.copy()
    arr_pieces = []
    pa = 0
    st = 0
    needdivcnt = 0
    for i, a in enumerate(arr):
        if a!=pa:
            arr_pieces.append([st, i])
            if (i-st)!=1: needdivcnt+=1
            pa = a
            st = i

    arr_pieces.append([st, len(arr)])
    if (len(arr)-st)!=1: needdivcnt+=1

    offset = range(len(arr_pieces), len(arr_pieces)+needdivcnt)
    p=0
    for arr_p in arr_pieces:
        length = arr_p[1] - arr_p[0]
        if length == 1: continue
        half_len = int(np.ceil(length / 2))
        for j in range(arr_p[0]+half_len, arr_p[1]):
            try:
                arr_res[j] = offset[p]
            except:
                print('wtf')
        p+=1
    return arr_res

def get_division_tree(n_agents):
    agent2divitreeindex = np.arange(n_agents)
    np.random.shuffle(agent2divitreeindex)
    max_div = np.ceil(np.log2(n_agents)).astype(int)
    levels = np.zeros(shape=(max_div+1, n_agents), dtype=int)
    tree_of_agent = []*(max_div+1)
    for ith, level in enumerate(levels):
        if ith == 0: continue
        res = _2div(levels[ith-1,:])
        levels[ith,:] = res
    res_levels = levels.copy()
    for i, div_tree_index in enumerate(agent2divitreeindex):
        res_levels[:, i] = levels[:, div_tree_index]
    return res_levels

In [29]:
from torch.distributions.categorical import Categorical
probs = torch.Tensor([
    [0.3, 0.3, 0.4],
    [0.2, 0.5, 0.3],
    [0.7, 0.2, 0.1],
    ])

q = [
    {0:0, 1:0, 2:0,},
    {0:0, 1:0, 2:0,},
    {0:0, 1:0, 2:0,},
]
yita = p_hit = 0.5
min_prob = 0.2


def random_process_with_clamp(probs, hit):
    with torch.no_grad():
        max_place = probs.argmax(-1, keepdims=True)
        mask_max = torch.zeros_like(probs).scatter_(-1, max_place, 1).bool()
        pmax = probs[mask_max].reshape(max_place.shape) #probs[max_place].clone()
        if hit:
            assert max_place.shape[-1] == 1
            return max_place.squeeze(-1)
        else:
            # forbit max prob being chosen
            # pmax = probs.max(axis=-1) #probs[max_place].clone()
            yita_arr = torch.ones_like(pmax)*yita
            yita_arr_clip = torch.minimum(pmax, yita_arr)
            # p_hat = pmax + (pmax-1) / (1/yita_arr_clip-1) + 1e-10
            p_hat = (pmax-yita_arr_clip)/(1-yita_arr_clip)
            k = 1/(1-yita_arr_clip)
            probs *= k
            probs[mask_max] = p_hat.reshape(-1)

            # print(probs)
            dist = Categorical(probs=probs)
            samp = dist.sample()
            assert samp.shape[-1] != 1
            return samp #.squeeze(-1)

def random_process_with_clamp2(probs, hit):
    with torch.no_grad():
        max_place = probs.argmax(-1, keepdims=True)
        mask_max = torch.zeros_like(probs).scatter_(-1, max_place, 1).bool()
        pmax = probs[mask_max].reshape(max_place.shape) #probs[max_place].clone()
        if hit:
            assert max_place.shape[-1] == 1
            return max_place.squeeze(-1)
        else:
            # forbit max prob being chosen
            # pmax = probs.max(axis=-1) #probs[max_place].clone()
            yita_arr = torch.ones_like(pmax)*yita
            # p_hat = pmax + (pmax-1) / (1/yita_arr_clip-1) + 1e-10
            p_hat = (pmax-yita_arr)/(1-yita_arr)
            p_hat = p_hat.clamp(min=min_prob)
            k = (1-p_hat)/(1-pmax)
            probs *= k
            probs[mask_max] = p_hat.reshape(-1)

            # print(probs)
            dist = Categorical(probs=probs)
            samp = dist.sample()
            assert samp.shape[-1] != 1
            return samp #.squeeze(-1)

# [0.3, 0.3, 0.4],
# [0.2, 0.5, 0.3],
# [0.7, 0.2, 0.1],

# tensor([[0.4500, 0.4500, 0.1000],
#         [0.3600, 0.1000, 0.5400],
#         [0.4000, 0.4000, 0.2000]])
for i in range(10000):
    hit = True if torch.rand(()) < p_hit else False
    res = random_process_with_clamp(probs.clone(), hit)
    res2 = random_process_with_clamp2(probs.clone(), hit)
    hit = True if torch.rand(()) < p_hit else False
    for j in range(3):
        q[j][res[j].item()] += 1
print(q)


[{0: 2475, 1: 2484, 2: 5041}, {0: 1938, 1: 5041, 2: 3021}, {0: 7031, 1: 1946, 2: 1023}]


In [16]:
import numpy as np 
n_agents = 10
seed = 0.4
probs_list = [0.1,0.2,0.7]
reward_which_act_list = [5,6,7]
def _schochestic_reward_(act, seed, probs_list, reward_which_act_list):
    s = 0
    ratio = 2
    for i in range(len(probs_list)):
        s += probs_list[i]
        if seed <= s:
            reward_which_act = reward_which_act_list[i]
            break
        
    return sum(act==reward_which_act)/(n_agents)*ratio

act = np.array([7,7,7,7,7,7,7,7,7,7])
act = np.array([7,7,7,7,7,7,7,7,7,7])*0 + 8
_schochestic_reward_(act, seed, probs_list, reward_which_act_list)

0.0

In [26]:
import torch
from torch.distributions.categorical import Categorical
from torch.nn.functional import kl_div
import torch.nn.functional as F

probs = torch.Tensor(
    [0.4, 0.6])
probs2 = torch.Tensor(
    [0.3, 0.7])

print(kl_div(probs.log(), probs2, reduction='sum'))
print(kl_div(probs2.log(), probs, reduction='sum'))
# print(kl_div(probs2.log(), probs, reduction='sum'))


tensor(0.0216)
tensor(0.0226)


client.send_targeted_dgram('ddd')

In [115]:
import torch.nn.functional as F
x = torch.Tensor(
    [0.4, 0.6])
y = torch.Tensor(
    [0.9, 0.1])
kl = F.kl_div(x.log(), y, reduction='sum')
kl  # kl 散度越大，说明概率分布的差异越大

tensor(0.5507)

In [19]:
x.softmax(dim=-1).log()

tensor([-0.7981, -0.5981])

In [54]:
import torch
from torch.distributions.categorical import Categorical
from torch.nn.functional import kl_div
import torch.nn.functional as F
from UTILS.tensor_ops import repeat_at
probs = torch.Tensor(
    [
        [0.4, 0.6],
        [0.3, 0.7],
        [0.3, 0.7]
    ]
    )


# probs.S # (?, n_agent, n_action)
n_agent = probs.shape[-2]
probs_rep = repeat_at(tensor=probs, insert_dim=-2, n_times=n_agent)
# probs_rep.S # (?, n_agent, n_agent, n_action)
probs_rep_transpose = probs_rep.swapaxes(-2,-3)
mat = (probs_rep*probs_rep.log()-probs_rep*probs_rep_transpose.log()).sum(-1)
mat # (?, n_agent, n_agent)
# F.kl_div(probs_rep.log(), probs_rep_transpose, reduction='batchmean')



tensor([[0.0000, 0.0226, 0.0226],
        [0.0216, 0.0000, 0.0000],
        [0.0216, 0.0000, 0.0000]])

In [110]:
# n_agent = 16
tree = get_division_tree(18)
# current_level = 3
# blood_distance = np.ones(shape=(n_agent,n_agent), dtype=np.float64) * np.nan
# for i in range(n_agent):
#     for j in range(n_agent):
#         if i==j:blood_distance[i,j] = 0
#         if not np.isnan(blood_distance[i,j]):
#             continue
#         for t in range(current_level+1):
#             investigate_level = (current_level) - t
#             if tree[investigate_level, i] == tree[investigate_level, j]:
#                 blood_distance[i,j] = t
#                 blood_distance[j,i] = t
#                 break


blood_distance = get_blood_distance(tree, 2)
print(blood_distance)
print(tree)

[[0. 2. 1. 1. 2. 1. 1. 2. 2. 2. 0. 2. 1. 0. 2. 0. 2. 2.]
 [2. 0. 2. 2. 1. 2. 2. 0. 0. 1. 2. 1. 2. 2. 0. 2. 1. 1.]
 [1. 2. 0. 0. 2. 0. 0. 2. 2. 2. 1. 2. 0. 1. 2. 1. 2. 2.]
 [1. 2. 0. 0. 2. 0. 0. 2. 2. 2. 1. 2. 0. 1. 2. 1. 2. 2.]
 [2. 1. 2. 2. 0. 2. 2. 1. 1. 0. 2. 0. 2. 2. 1. 2. 0. 0.]
 [1. 2. 0. 0. 2. 0. 0. 2. 2. 2. 1. 2. 0. 1. 2. 1. 2. 2.]
 [1. 2. 0. 0. 2. 0. 0. 2. 2. 2. 1. 2. 0. 1. 2. 1. 2. 2.]
 [2. 0. 2. 2. 1. 2. 2. 0. 0. 1. 2. 1. 2. 2. 0. 2. 1. 1.]
 [2. 0. 2. 2. 1. 2. 2. 0. 0. 1. 2. 1. 2. 2. 0. 2. 1. 1.]
 [2. 1. 2. 2. 0. 2. 2. 1. 1. 0. 2. 0. 2. 2. 1. 2. 0. 0.]
 [0. 2. 1. 1. 2. 1. 1. 2. 2. 2. 0. 2. 1. 0. 2. 0. 2. 2.]
 [2. 1. 2. 2. 0. 2. 2. 1. 1. 0. 2. 0. 2. 2. 1. 2. 0. 0.]
 [1. 2. 0. 0. 2. 0. 0. 2. 2. 2. 1. 2. 0. 1. 2. 1. 2. 2.]
 [0. 2. 1. 1. 2. 1. 1. 2. 2. 2. 0. 2. 1. 0. 2. 0. 2. 2.]
 [2. 0. 2. 2. 1. 2. 2. 0. 0. 1. 2. 1. 2. 2. 0. 2. 1. 1.]
 [0. 2. 1. 1. 2. 1. 1. 2. 2. 2. 0. 2. 1. 0. 2. 0. 2. 2.]
 [2. 1. 2. 2. 0. 2. 2. 1. 1. 0. 2. 0. 2. 2. 1. 2. 0. 0.]
 [2. 1. 2. 2. 0. 2. 2. 1. 1. 0.

In [72]:

[[ 0.  2.  2. nan  2. nan  2. nan nan  2. nan  2. nan nan  2. nan]
 [ 2.  0.  2. nan  2. nan  2. nan nan  2. nan  2. nan nan  2. nan]
 [ 2.  2.  0. nan  2. nan  2. nan nan  2. nan  2. nan nan  2. nan]
 [nan nan nan  0. nan  2. nan  2.  2. nan  2. nan  2.  2. nan  2.]
 [ 2.  2.  2. nan  0. nan  2. nan nan  2. nan  2. nan nan  2. nan]
 [nan nan nan  2. nan  0. nan  2.  2. nan  2. nan  2.  2. nan  2.]
 [ 2.  2.  2. nan  2. nan  0. nan nan  2. nan  2. nan nan  2. nan]
 [nan nan nan  2. nan  2. nan  0.  2. nan  2. nan  2.  2. nan  2.]
 [nan nan nan  2. nan  2. nan  2.  0. nan  2. nan  2.  2. nan  2.]
 [ 2.  2.  2. nan  2. nan  2. nan nan  0. nan  2. nan nan  2. nan]
 [nan nan nan  2. nan  2. nan  2.  2. nan  0. nan  2.  2. nan  2.]
 [ 2.  2.  2. nan  2. nan  2. nan nan  2. nan  0. nan nan  2. nan]
 [nan nan nan  2. nan  2. nan  2.  2. nan  2. nan  0.  2. nan  2.]
 [nan nan nan  2. nan  2. nan  2.  2. nan  2. nan  2.  0. nan  2.]
 [ 2.  2.  2. nan  2. nan  2. nan nan  2. nan  2. nan nan  0. nan]
 [nan nan nan  2. nan  2. nan  2.  2. nan  2. nan  2.  2. nan  0.]]

 

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  1,  0,  1,  1,  1,  1,  0,  1,  0,  0,  0,  1,  0,  0],
       [ 1,  0,  3,  2,  3,  1,  1,  1,  2,  3,  0,  2,  2,  3,  0,  0],
       [ 1,  0,  3,  5,  7,  6,  1,  6,  2,  7,  4,  5,  2,  3,  4,  0],
       [12,  0,  3, 11, 15,  6,  1, 13, 10,  7,  9,  5,  2, 14,  4,  8]])

In [2]:
import pickle
with open('RECYCLE/wifi', 'rb') as f:
    p_list = pickle.load(f)

p_list

{'d1': tensor([1.1618e-13, 4.3868e-16, 3.4678e-18, 0.0000e+00, 1.0000e+00, 8.3701e-14,
         1.8273e-12], device='cuda:0', requires_grad=True),
 'd2': tensor([0.0756, 0.2109, 0.0628, 0.0104, 0.3506, 0.1273, 0.1624],
        device='cuda:0', requires_grad=True)}

In [32]:
import torch
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.nn.functional import kl_div


logits1 = torch.Tensor([  4.4786,  -1.1005,  -5.9407, -15.7646,  5.2623,   4.1508,   7.2341]).type(torch.DoubleTensor)
logits2 = torch.Tensor([ -0.5480,  0.4779, -0.7330, -2.5290,  0.9862, -0.0273,  0.2163]).type(torch.DoubleTensor)

dist1 = Categorical(logits = logits1)
dist2 = Categorical(logits = logits2)
print(dist1.probs)
print(dist2.probs)
from torch.distributions import kl_divergence

print(kl_divergence(dist1, dist2))
print(kl_divergence(dist2, dist1))


tensor([5.0909e-02, 1.9223e-04, 1.5197e-06, 8.2279e-11, 1.1147e-01, 3.6681e-02,
        8.0075e-01], dtype=torch.float64)
tensor([0.0756, 0.2109, 0.0628, 0.0104, 0.3506, 0.1273, 0.1624],
       dtype=torch.float64)
tensor(1.0829, dtype=torch.float64)
tensor(2.6698, dtype=torch.float64)


In [7]:
# probs = torch.Tensor(
#     [0, 1])
# probs2 = torch.Tensor(
#     [0.3, 0.7])

# probs = p_list['d1']
# probs2 = p_list['d2']

# print(kl_div(probs.log(), probs2))
# print(kl_div(probs2.log(), probs))

# probs = torch.Tensor(
#     [0, 1])
# probs2 = torch.Tensor(
#     [0.3, 0.7])

# probs = p_list['d1']
# probs2 = p_list['d2']

# print(kl_div(probs.log(), probs2))
# print(kl_div(probs2.log(), probs))

# probs = torch.Tensor(
#     [0, 1])
# probs2 = torch.Tensor(
#     [0.3, 0.7])

# probs = p_list['d1']
# probs2 = p_list['d2']

# print(kl_div(probs.log(), probs2))
# print(kl_div(probs2.log(), probs))

# probs = torch.Tensor(
#     [0, 1])
# probs2 = torch.Tensor(
#     [0.3, 0.7])

# probs = p_list['d1']
# probs2 = p_list['d2']

# print(kl_div(probs.log(), probs2))
# print(kl_div(probs2.log(), probs))

tensor([1.1618e-13, 4.3868e-16, 3.4678e-18, 0.0000e+00, 1.0000e+00, 8.3701e-14,
        1.8273e-12], device='cuda:0', requires_grad=True)

In [64]:
import torch
import numpy as np
from UTILS.tensor_ops import gather_righthand, repeat_at
def scatter_righthand(scatter_into, src, index, check=True):
    index = index.long()
    i_dim = index.dim()
    s_dim = src.dim()
    t_dim = i_dim - 1
    index_new_shape = list(src.shape)
    index_new_shape[t_dim] = index.shape[t_dim]
    for _ in range(i_dim, s_dim):
        index = index.unsqueeze(-1)
    index_expand = index.expand(index_new_shape)  # only this two line matters
    return scatter_into.scatter(t_dim, index_expand, src)

orig = torch.Tensor([[[ 0,  1,  2], [ 3,  4,  5]],
                    [[ 6,  7,  8], [ 9, 10, 11]],
                    [[12, 13, 14], [15, 16, 17]]])
index = torch.Tensor([[0], [1], [0]])
print(orig)

res      = gather_righthand(src=orig, index=index)
res[:] = 0

orig_fix = orig.clone().detach()
orig_fix = scatter_righthand(orig_fix, src=res, index=index)
print(orig_fix)


# orig.scatter(dim=1,index=index.long(),src=res)

tensor([[[ 0.,  1.,  2.],
         [ 3.,  4.,  5.]],

        [[ 6.,  7.,  8.],
         [ 9., 10., 11.]],

        [[12., 13., 14.],
         [15., 16., 17.]]])
tensor([[[ 0.,  0.,  0.],
         [ 3.,  4.,  5.]],

        [[ 6.,  7.,  8.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [15., 16., 17.]]])


In [71]:
index = torch.randint(high=16,size=(64,5))
print('s1')
res = gather_righthand(src,index)
print('s1')
res[:] = 0
print('s2')
resX = scatter_righthand(scatter_into=src, src=res, index=index)
print('s2')
print(resX.S)


s1
s1
s2
s2
torch.Size([64, 16, 8, 88, 888])


In [3]:
import torch
import numpy as np
from UTILS.tensor_ops import gather_righthand, repeat_at
from UTILS.tensor_ops import add_onehot_id_at_last_dim, repeat_at, _2tensor, gather_righthand, scatter_righthand
x_in = torch.Tensor([
    [1,1,1],
    [2,2,2],
    [3,3,3],
    [4,4,4],
])
n_agent = 4


nets = [
    lambda x: x+100,
    lambda x: x+200,
    lambda x: x+300,
    lambda x: x+400
]


x0 = add_onehot_id_at_last_dim(x_in)
# x1 = self.shared_net(x0)
res = []
for i in range(n_agent):
    res.append(nets[i](x0[..., i, :]))
x2 = torch.stack(res, -2)
# x22 = self.nets[0](x1)
print('x2',x2)

######### forward twice: shuffle forward
perm_index = torch.randperm(n_agent, device=x_in.device)   # shape = (n_agent)
print('perm_index',perm_index)

perm_index = perm_index.expand(x_in.shape[:-1]) # shape = (...?, n_agent)
# x_in shape = (...?, n_agent, coredim)
perm_x_in = gather_righthand(src=x_in, index=perm_index, check=False)
perm_x0 = add_onehot_id_at_last_dim(perm_x_in)
perm_res = []
for i in range(n_agent):
    perm_res.append(nets[i](perm_x0[..., i, :]))
perm_x2 = torch.stack(perm_res, -2)

# 103
# 202
# 304
# 401


# 401 vs 101
# 202 vs 202
# 103 vs 303
# 304 vs 404

confact_x2 = torch.zeros_like(perm_x2) + np.nan
confact_x2 = scatter_righthand(scatter_into=confact_x2, src=perm_x2, index=perm_index)
print('confact_x2',confact_x2)

x2 tensor([[101., 101., 101., 101., 100., 100., 100.],
        [202., 202., 202., 200., 201., 200., 200.],
        [303., 303., 303., 300., 300., 301., 300.],
        [404., 404., 404., 400., 400., 400., 401.]])
perm_index tensor([2, 1, 3, 0])
confact_x2 tensor([[401., 401., 401., 400., 400., 400., 401.],
        [202., 202., 202., 200., 201., 200., 200.],
        [103., 103., 103., 101., 100., 100., 100.],
        [304., 304., 304., 300., 300., 301., 300.]])


In [4]:
import torch

res=torch.isnan(torch.tensor([1, float('inf'), 2, float('-inf'), float('nan')]))
print(res)

res=torch.isfinite(torch.tensor([1, float('inf'), 2, float('-inf'), float('nan')]))
print(res)







tensor([False, False, False, False,  True])
tensor([ True, False,  True, False, False])
